In [ ]:
# GET POR Files Already Downloaded
import glob
import shutil

source = '/Users/kris/Downloads/FFIEC/'
destination = '/Users/kris/Library/CloudStorage/OneDrive-Personal/Documents/Visual Studio 2019/Repos/ffiec-xbrl/PubData/Src/RSSDs/'

for src in glob.glob(source + '*/* POR *'):
    shutil.copy(src, destination)

In [1]:
# Imports and Constants
import pandas as pd
import numpy as np

#dtypes = {'RSSD_ID':str,
#          'FDIC_Certificate_Number':str,
#          'OCC_Charter_Number':str,
#          'OTS_Docket_Number':str,
#          'Primary_ABA_Routing_Number':str,
#          'Financial_Institution_Name': str,
#          'Financial_Institution_Address': str,
#          'Financial_Institution_City': str,
#          'Financial_Institution_State': str,
#          'Financial_Institution_Zip_Code': str,
#          'Financial_Institution_Filing_Type': str,
#          'Last Date/Time Submission Updated On': 'str'}

dtypes = {'IDRSSD': np.dtype('str'),
          'FDIC Certificate Number': np.dtype('str'),
          'OCC Charter Number': np.dtype('str'),
          'OTS Docket Number': np.dtype('str'),
          'Primary ABA Routing Number': np.dtype('str'),
          'Financial Institution Name': np.dtype('str'),
          'Financial Institution Address': np.dtype('str'),
          'Financial Institution City': np.dtype('str'),
          'Financial Institution State': np.dtype('str'),
          'Financial Institution Zip Code': np.dtype('str'),
          'Financial Institution Filing Type': np.dtype('str'),
          'Last Date/Time Submission Updated On': np.dtype('str')}

filepath_in = 'PubData/Src/RSSDs/'
filepath_out = 'PubData/'
filename = 'FFIEC CDR Call Bulk POR 03312024.txt'

In [ ]:
# Compare Bank_Dim and RSSD_Dict
df_bank = pd.read_csv(filepath_out + 'Bank_Dim.csv', sep=',', index_col=False, quotechar='"')
df_rssd = pd.read_csv(filepath_out + 'RSSD_Dict.csv', sep=',', index_col=False, quotechar='"')

print(df_bank.shape, df_rssd.shape)


In [2]:
#Open File (Bank_dim from RSSD_Por)

#with open(filename, 'r') as file:
df = pd.read_csv(filepath_in + filename, sep='\t', index_col=False, quotechar='"', dtype = dtypes, parse_dates=['Last Date/Time Submission Updated On'])

In [3]:
#Clean RSSD_Por File
df = df.apply(lambda x: x.str.strip() if x.dtype.name == 'object' else x, axis=0)

In [4]:
#Clean RSSD_Por Format - Align with and Create RSSD_Dict.csv
df.rename(columns={'IDRSSD': 'RSSD_ID'}, inplace=True)
df.columns = df.columns.str.replace(' ', '_')

#Create RSSD_Dict file
#df.iloc[:25].to_csv(filepath_out + 'RSSD_Dict.csv', sep=',', quotechar='"', index= False,)
df.to_csv(filepath_out + 'RSSD_Dict.csv', sep=',', quotechar='"', index= False)

In [ ]:
#Confirm data
df[:5]

#df.Financial_Institution_Address
df.iloc[0,6]

In [5]:
from FDIC import ExEngine
from FDIC.ETL import ETL, RateLimiter
from FDIC import constants as paths

rate_limiter = RateLimiter(max_calls=10, period_in_seconds=40)
#rate_limiter = RateLimiter(max_calls=1000, period_in_seconds=3660)
etl = ETL(paths.WSDL_path, rate_limiter)
#ExEngine.FillMaster('504713')
#etl.GenBankDim()
etl.DownloadCallReports(['XBRL'])

#etl.ParseXBRL()
#etl.GenBankMaster()
#etl.GenCallMaster()

Already downloaded: BANK OF HANCOCK COUNTY_37
Already downloaded: FIRST COMMUNITY BANK XENIAFLORA_242
Already downloaded: BROADSTREET BANK SSB_279
Already downloaded: BISON STATE BANK_354
Already downloaded: LOWRY STATE BANK_457
Already downloaded: BALLSTON SPA NATIONAL BANK_505
Already downloaded: RVR BANK_1155
Already downloaded: BANK OF GRANDIN_1351
Already downloaded: HILLTOP NATIONAL BANK_1454
Already downloaded: LIBERTY NATIONAL BANK IN PARIS THE_1669
Already downloaded: WASHITA STATE BANK_1856
Already downloaded: LAMAR NATIONAL BANK_2161
Already downloaded: ALBANY BANK  TRUST CO NATIONAL ASSOCIATION_2732
Already downloaded: GENOA COMMUNITY BANK_2750
Already downloaded: FIRST NATIONAL BANK OF GORDON THE_3252
Already downloaded: FARMBANK_3458
Already downloaded: BANK OF CLARENDON THE_3720


UnboundLocalError: local variable 'skipped_dtypes' referenced before assignment

In [ ]:
call_report_path = '/Users/kris/data_sources/fdic/BulkReports/ACNB BANK_5210/ACNB BANK_5210_20010331.XBRL'
bank_path = '/Users/kris/data_sources/fdic/BulkReports/ACNB BANK_5210/'
folder_path = '/Users/kris/data_sources/fdic/BulkReports/'

#tot = etl.ParseXBRL(call_report_path)
etl.GenBankMaster()
#etl.GenCallMaster()

In [ ]:
etl.GenCallMaster()

In [ ]:
import FDIC.constants as paths
import pandas as pd
#tot

df = pd.read_csv(paths.localPath + paths.filename_MasterCall)


In [ ]:
df.head(1)

In [ ]:
#Number of RSSD's by Period, by RSSD_ID, by MDRM_Item
g_df = df[df['RSSD_ID']==28013].groupby(['MDRM_Item', 'RSSD_ID', 'ReportPeriodEndDate'])['MDRM_Item'].count()
g_df

In [ ]:
#Number of RSSD's by field by date
pvt = df[['MDRM_Item', 'ReportPeriodEndDate']].pivot_table(
    index=['MDRM_Item'],
    columns=['ReportPeriodEndDate'],
    values=['MDRM_Item'],
    aggfunc={'MDRM_Item': ['count']}
)
pvt.fillna(0, inplace=True)
pvt

In [ ]:
#Number of RSSD's by date
pvt = df[['ReportPeriodEndDate', 'RSSD_ID']].drop_duplicates().reset_index().pivot_table(
    index=['ReportPeriodEndDate'],
    values=['RSSD_ID'],
    aggfunc={'RSSD_ID': ['count']}
).sort_values(by='ReportPeriodEndDate', ascending=False)

pvt = pvt.set_axis([col for col in pvt.keys()], axis=1)
#pvt.reset_index(inplace=True)

pvt.fillna(0, inplace=True)
pvt

In [ ]:
#df
grouped_df = df[['MDRM_Item', 'Item_Name']].drop_duplicates().groupby(['MDRM_Item', 'Item_Name']).size().reset_index(name='Count')
print(grouped_df)